<a href="https://colab.research.google.com/github/DyanielCX/DLI-Assm/blob/main/GuoXi_LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import All Dependencies & Libraries

In [11]:
import os, time, json, random
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility
np.random.seed(42); random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, callbacks, optimizers, Input

## Load Dataset

In [12]:
# ---------- 1) Load dataset (GitHub) ----------
GITHUB_CSV = "https://raw.githubusercontent.com/DyanielCX/DLI-Assm/refs/heads/main/dataset_B_05_2020_1.csv"
raw = pd.read_csv(GITHUB_CSV)

# Keep text + label for DL
df = raw[["url", "status"]].dropna().copy()
df["url"] = df["url"].astype(str).str.lower()
y = df["status"].astype(int).values
texts = df["url"].tolist()

# Character-level tokenization




In [13]:
tok = Tokenizer(char_level=True, lower=True, filters="")
tok.fit_on_texts(texts)

seqs = tok.texts_to_sequences(texts)
lengths = np.array([len(s) for s in seqs])

# Use 95th percentile as maxlen to avoid over-padding
maxlen = int(np.percentile(lengths, 95))
maxlen = max(maxlen, 64)          # safety floor
print(f"Max sequence length (95th pct): {maxlen}")

X = pad_sequences(seqs, maxlen=maxlen, padding="post", truncating="post")

Max sequence length (95th pct): 131


# Train/Val/Test split





In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.10, random_state=42, stratify=y_train
)

vocab_size = len(tok.word_index) + 1
print("Vocab size (chars):", vocab_size)

Vocab size (chars): 74


# Build LSTM model

In [15]:
def build_model(vocab, seq_len, emb_dim=64, lstm_units=64, lr=1e-3):
    inp = layers.Input(shape=(seq_len,), dtype="int32")
    x = layers.Embedding(vocab, emb_dim, input_length=seq_len)(inp)
    x = layers.Bidirectional(layers.LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(1, activation="sigmoid")(x)
    model = models.Model(inp, out)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

model = build_model(vocab_size, maxlen)

es = callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
rlr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(





# Train + measure time




In [ ]:
t0 = time.perf_counter()
hist = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=256,
    callbacks=[es, rlr],
    verbose=1
)
t1 = time.perf_counter()
train_time_minutes = (t1 - t0) / 60.0

Epoch 1/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 37s 672ms/step - accuracy: 0.6190 - loss: 0.6631 - val_accuracy: 0.7825 - val_loss: 0.5049 - learning_rate: 0.0010
Epoch 2/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 23s 689ms/step - accuracy: 0.7551 - loss: 0.5302 - val_accuracy: 0.7727 - val_loss: 0.4868 - learning_rate: 0.0010
Epoch 3/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 695ms/step - accuracy: 0.7663 - loss: 0.5083 - val_accuracy: 0.7814 - val_loss: 0.4767 - learning_rate: 0.0010
Epoch 4/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 40s 669ms/step - accuracy: 0.7741 - loss: 0.4946 - val_accuracy: 0.7880 - val_loss: 0.4709 - learning_rate: 0.0010
Epoch 5/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 42s 712ms/step - accuracy: 0.7757 - loss: 0.4812 - val_accuracy: 0.7902 - val_loss: 0.4479 - learning_rate: 0.0010
Epoch 6/50
 8/33 ━━━━━━━━━━━━━━━━━━━━ 17s 682ms/step - accuracy: 0.7856 - loss: 0.4621

# Evaluate

In [ ]:
t2 = time.perf_counter()
y_proba = model.predict(X_test, batch_size=512).ravel()
t3 = time.perf_counter()
pred_time_ms_per_sample = ((t3 - t2) / len(X_test)) * 1000.0

y_pred = (y_proba >= 0.5).astype(int)

acc  = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec  = recall_score(y_test, y_pred, zero_division=0)
f1   = f1_score(y_test, y_pred, zero_division=0)
auc  = roc_auc_score(y_test, y_proba)

print("\n=== LSTM Test Metrics ===")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"ROC-AUC  : {auc:.4f}")
print(f"Train time (min)        : {train_time_minutes:.2f}")
print(f"Prediction time (ms/sample): {pred_time_ms_per_sample:.2f}\n")
print("=== Classification Report ===")
print(classification_report(y_test, y_pred, digits=4))



# Plot Confusion matrix (normalized) and Roc Curve


In [ ]:
cm = confusion_matrix(y_test, y_pred, normalize="true")
plt.figure()
plt.imshow(cm, interpolation="nearest")
plt.title("Normalized Confusion Matrix — LSTM")
plt.xlabel("Predicted")
plt.ylabel("True")
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, f"{cm[i, j]:.2f}", ha="center", va="center")
plt.tight_layout()
plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
plt.figure()
plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.title("ROC Curve — LSTM")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()


# Save artifacts


In [ ]:
OUT = Path("outputs_lstm"); OUT.mkdir(exist_ok=True, parents=True)
model.save(OUT / "lstm_url_classifier.h5")

# Save tokenizer
with open(OUT / "tokenizer_charlevel.json", "w") as f:
    f.write(tok.to_json())

# Save metrics
pd.DataFrame([{
    "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-score": f1,
    "ROC-AUC": auc,
    "Training Time (minutes)": train_time_minutes,
    "Prediction Time (milliseconds)": pred_time_ms_per_sample,
    "Maxlen": maxlen, "VocabSize": vocab_size
}]).to_csv(OUT / "metrics_summary.csv", index=False)


# Compare With Researcher Model

In [ ]:
RP = {
    "Accuracy": 0.93,
    "Precision": 0.87,
    "Recall": 0.93,
    "F1-score": 0.89,
    "Training Time (minutes)": 2.64,
    "Prediction Time (milliseconds)": 14.00
}
my_vals = [acc, prec, rec, f1, train_time_minutes, pred_time_ms_per_sample]
rp_vals = [RP["Accuracy"], RP["Precision"], RP["Recall"], RP["F1-score"],
           RP["Training Time (minutes)"], RP["Prediction Time (milliseconds)"]]
titles = ["Accuracy","Precision","Recall",
          "F1-score","Training Time (minutes)","Prediction Time (milliseconds)"]

fig, axes = plt.subplots(2,3, figsize=(12,7)); axes = axes.ravel()
for i, ax in enumerate(axes):
    ax.bar(["My Model","Researcher"], [my_vals[i], rp_vals[i]])
    ax.set_title(titles[i]); ax.set_xlabel("Model")
    if i <= 3:
        ax.set_ylim(0, 1.05); ax.set_ylabel(titles[i])
    elif i == 4:
        ax.set_ylabel("Time (minutes)")
    else:
        ax.set_ylabel("Time (milliseconds)")
    for j, v in enumerate([my_vals[i], rp_vals[i]]):
        ax.text(j, v, f"{v:.2f}", ha="center", va="bottom")
plt.tight_layout()
plt.savefig(OUT / "comparison_bars.png", dpi=180)
plt.show()

print(f"Saved model & files to: {OUT.resolve()}")